In [5]:
!pip install ipywidgets

In [2]:
!pip install kagglehub

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/new-dataset-for-text-classification-ag-news")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/thedevastator/new-dataset-for-text-classification-ag-news/versions/2


In [4]:
import pandas as pd # для обработки и анализа данных
import numpy as np # для математических вычислений
from tqdm.auto import tqdm # для отслеживания прогресса длительных операций
import matplotlib.pyplot as plt # для визуализации


In [10]:
data_train = pd.read_csv('data/train.csv', sep=',')
data_test = pd.read_csv('data/test.csv', sep=',')


In [13]:
data_train['label'].unique()

array([2, 3, 1, 0])

In [14]:
labels_train = data_train['label']
labels_test = data_test['label']

len(labels_train)
# Output:
# 120000


120000